# Analysing an OpenSea NFT on the Polygon blockchain

Mal Minhas v0.1 21.02.22

### 1. Context and Setup

I recently bought an NFT on OpenSea using some ETH (Polygon) that I had in my linked MetaMask wallet. 
The NFT I bought is called Ask Me Later and is visible on OpenSea [here](https://opensea.io/assets/matic/0x2953399124f0cbb46d2cbacd8a89cf0599974963/86520523391455217104303705677473463659385359901978646282133996067748319330305).
This notebook uses Python `Web3` to develop client code that can interact with the Ethereum network to read data from the smart contract in the NFT and allow me to inspect what's in the NFT.

In order to get going we need to set up a virtual environment which in this instance we'll call `web3` and then `pip install` what we need into it.  The following assumes you have set up the virtual environment already:

```
(web3) $ pip install web3, requests
```

### 2. Connecting to the Ethereum Network

The contract for the NFT we are interested in is available on Polygon [here](https://polygonscan.com/address/0x2953399124f0cbb46d2cbacd8a89cf0599974963).  We reach that link by clicking on the corresponding contract link on the OpenSea NFT page [here](https://opensea.io/assets/matic/0x2953399124f0cbb46d2cbacd8a89cf0599974963/86520523391455217104303705677473463659385359901978646282133996067748319330305).  We can connect to the Ethereum network via an HTTP provider courtesy of Polygon.  The location of that provider was picked up from [this Reddit post](https://www.reddit.com/r/0xPolygon/comments/q0tt61/how_to_interact_with_polygon_using_web3py/).

We can read smart contracts with the Python `web3` module.  In order to do that we need:
* An ABI (or Abstract Binary Interface) specification of the functions on the smart contract.
* A Python representation of the smart contract which is done using a call to `web3.eth.Contract()` 

The `web3.eth.Contract()` call takes two arguments: the smart contract ABI and the smart contract address.  An ABI in this context is a JSON array that describes how a specific smart contract works. The guide [here](https://www.dappuniversity.com/articles/web3-py-intro) is very useful for understanding how to practically use `web3` with smart contracts. 

In the following code, we will start with just two simple ABI functions inherited from the original [ERC-721](https://eips.ethereum.org/EIPS/eip-721) NFT specification, namely `symbol` and `name`:

In [16]:
from web3 import Web3, HTTPProvider

my_abi = [
    {# OK
        'inputs': [],
        'name': 'name',
        'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
        'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {# OK
        'inputs': [],
        'name': 'symbol',
        'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
        'stateMutability': 'view', 'type': 'function', 'constant': True
    },
]

uri = "https://polygon-rpc.com/"
w3 = Web3(Web3.HTTPProvider(endpoint_uri=uri, request_kwargs={'timeout': 10}))
print(f'web3 connection status to HTTP provider at {uri} = {w3.isConnected()}')
my_token_addr = "0x2953399124f0cbb46d2cbacd8a89cf0599974963"

# See here for explanation of checksum address: 
# https://coincodex.com/article/2078/ethereum-address-checksum-explained/
contract = w3.eth.contract(address=w3.toChecksumAddress(my_token_addr), abi=my_abi)
print(f'contract address: {contract.address}')
print(f'contract functions: {contract.all_functions()}')
name = contract.functions.name().call()
symbol = contract.functions.symbol().call()
print(f"{name} [{symbol}]")

web3 connection status to HTTP provider at https://polygon-rpc.com/ = True
contract address: 0x2953399124F0cBB46d2CbACD8A89cF0599974963
contract functions: [<Function name()>, <Function symbol()>]
OpenSea Collections [OPENSTORE]


Now we have the basic setup in place we can look to get further into the ABI.

### 3. Mapping the ABI

In order to get further information from the contract, we need to understand how the contract is constructed in greater detail.  The contract is written in [Solidity](https://en.wikipedia.org/wiki/Solidity). The corresponding Solidity code for the smart contract is available for inspection on Polygon [here](https://polygonscan.com/address/0x2953399124f0cbb46d2cbacd8a89cf0599974963#code).  It is built using something called [ERC-1155](https://eips.ethereum.org/EIPS/eip-1155) which is a "next-generation" multi-token standard backwards compatible with the original [ERC-721](https://eips.ethereum.org/EIPS/eip-721) NFT specification.

In [18]:
my_abi2 = [
    {
        'inputs': [{'internalType': 'address', 'name': 'owner', 'type': 'address'},
                   {'internalType': 'uint256', 'name': 'id', 'type': 'uint256'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'payable': False, 'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {
        'inputs': [{'internalType': 'uint256', 'name': 'id', 'type': 'uint256'}],
        'name': 'creator',
        'outputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'payable': False, 'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {
        'inputs': [{'internalType': 'uint256', 'name': 'id', 'type': 'uint256'}],
        'name': 'isPermanentURI',
        'outputs': [{'internalType': 'bool', 'name': '', 'type': 'bool'}],
        'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {
        'inputs': [],
        'name': 'name',
        'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
        'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {
        'inputs': [],
        'name': 'openSeaVersion',
        'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
        'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {
        'inputs': [],
        'name': 'owner',
        'outputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'payable': False, 'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {
        'inputs': [],
        'name': 'symbol',
        'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
        'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {
        'inputs': [],
        'name': 'templateURI',
        'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
        'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {
        'inputs': [{'internalType': 'uint256', 'name': 'id', 'type': 'uint256'}],
        'name': 'maxSupply',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {
        'inputs': [{'internalType': 'uint256', 'name': 'id', 'type': 'uint256'}],
        'name': 'totalSupply',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view', 'type': 'function', 'constant': True
    },
    {
        'inputs': [{'internalType': 'uint256', 'name': 'id', 'type': 'uint256'}],
        'name': 'uri',
        'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
        'stateMutability': 'view', 'type': 'function', 'constant': True
    },
]

Now we have a fully specified ABI, we can start again 

In [20]:
my_token_id = 86520523391455217104303705677473463659385359901978646282133996067748319330305

print(f"------ 2. Working with the smart contract ------")
contract = w3.eth.contract(address=w3.toChecksumAddress(my_token_addr), abi=my_abi2)
print(f'contract address: {contract.address}')
print(f'contract functions: {contract.all_functions()}')
name = contract.functions.name().call()
symbol = contract.functions.symbol().call()
version = contract.functions.openSeaVersion().call()
owner_acc = contract.functions.owner().call()
print(f"{name} v{version} [{symbol}]: owner={owner_acc}")
print(f"------ 2. Working with the NFT ------")
templateURI = contract.functions.templateURI().call()
print(f"templateURI: {templateURI}")
uri = contract.functions.uri(my_token_id).call()
assert(uri == templateURI)
nftURI = uri.replace('0x{id}', str(my_token_id))
print(f"NFT URI: {nftURI}")
isPermanentURI = contract.functions.isPermanentURI(my_token_id).call()
print(f"is a permanent URI = {isPermanentURI}")
balanceOfOwner = contract.functions.balanceOf(owner=owner_acc,id=my_token_id).call()
print(f"balanceOf owner = {balanceOfOwner}")
totalSupply = contract.functions.totalSupply(my_token_id).call()
print(f"total supply = {totalSupply}")
maxSupply = contract.functions.maxSupply(my_token_id).call()
print(f"max supply = {maxSupply}")
creator_acc = contract.functions.creator(my_token_id).call()
print(f"creator = {creator_acc}")
balanceOfCreator = contract.functions.balanceOf(owner=creator_acc, id=my_token_id).call()
print(f"balanceOf creator = {balanceOfCreator}")

------ 2. Working with the smart contract ------
contract address: 0x2953399124F0cBB46d2CbACD8A89cF0599974963
contract functions: [<Function balanceOf(address,uint256)>, <Function creator(uint256)>, <Function isPermanentURI(uint256)>, <Function name()>, <Function openSeaVersion()>, <Function owner()>, <Function symbol()>, <Function templateURI()>, <Function maxSupply(uint256)>, <Function totalSupply(uint256)>, <Function uri(uint256)>]
OpenSea Collections v2.1.0 [OPENSTORE]: owner=0x5b3256965e7C3cF26E11FCAf296DfC8807C01073
------ 2. Working with NFT ------
templateURI: https://api.opensea.io/api/v2/metadata/matic/0x2953399124F0cBB46d2CbACD8A89cF0599974963/0x{id}
NFT URI: https://api.opensea.io/api/v2/metadata/matic/0x2953399124F0cBB46d2CbACD8A89cF0599974963/86520523391455217104303705677473463659385359901978646282133996067748319330305
is a permanent URI = False
balanceOf owner = 0
total supply = 1
max supply = 1
creator = 0xBF48E17f3E2889Da0DEC0af8B4fA768dBCf236D3
balanceOf creator = 0


A couple of interesting things here are that the OpenSea URI is designated as `False` so OpenSea presumably can change this.  Also that as expected for an NFT, `totalSupply` is 1.

### 4. Downloading the NFT

Now that we have our NFT URI in the `nftURI` variable, we can finally attempt to download it to see what's in the box:

In [21]:
import requests, pprint

r = requests.get(nftURI)
pprint.pprint(r.json())

{'animation_url': None,
 'description': 'Ye reflecting on NFTs and the permanence within',
 'external_link': None,
 'image': 'https://lh3.googleusercontent.com/IOBYa98fiHX2fbW8-9wm_h2ZBKk06uqPen9wr0eBCG8doxACDN37VCxxbeN28vIaWmX52waQ5fNIQNVPIk1Y6lnsehBPxAK3eA7lEQ',
 'name': 'Ask Me Later',
 'traits': []}


Wow.  The image returned in the NFT URI is nothing other than a link to a thumbnail on Google.  It seems you only get the high fidelity image within the OpenSea environment.

In [28]:
from IPython.display import Image

Image(url= "https://lh3.googleusercontent.com/IOBYa98fiHX2fbW8-9wm_h2ZBKk06uqPen9wr0eBCG8doxACDN37VCxxbeN28vIaWmX52waQ5fNIQNVPIk1Y6lnsehBPxAK3eA7lEQ", width=1200, height=1200)